In [ ]:
!pip3 install matplotlib

In [10]:
!pip3 install kagglehub


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.13 -m pip install --upgrade pip


In [9]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [10]:
import kagglehub

path = kagglehub.dataset_download("oddrationale/mnist-in-csv")

print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/anushkasharma/.cache/kagglehub/datasets/oddrationale/mnist-in-csv/versions/2


In [11]:
data = pd.read_csv("/Users/anushkasharma/.cache/kagglehub/datasets/oddrationale/mnist-in-csv/versions/2/mnist_train.csv")

In [12]:
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data = np.array(data)
m,n=data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train =data_train[1:n]


In [17]:
X_train[:, 0].shape

(784,)

In [31]:
#Initialize parameters

from numpy import exp


def init_params():
    w1=np.random.rand(10, 784) - 0.5
    b1=np.random.rand(10,1) - 0.5
    w2=np.random.rand(10, 10) - 0.5
    b2=np.random.rand(10,1) - 0.5
    return w1, b1, w2, b2

def ReLU(Z):
    return np.maximum(Z,0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

#FORWARD PROPAGATION
def forward_prop (w1, b1, w2, b2, X):
    Z1 = w1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = w2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y =np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_reLU(Z):
    return Z>0


#Backward Propagation

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_reLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2


def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha*dw1
    b1 = b1 - alpha*db1
    w2 = w2 - alpha*dw2
    b2 = b2 - alpha*db2
    return w1, b1, w2, b2
    

In [34]:
def get_predictions(A2):
    return np.argmax(A2,0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

/var/folders/fy/vqqz7dk157x6lpsmjr653xrh0000gn/T/ipykernel_6542/2472023414.py:17: RuntimeWarning: overflow encountered in exp
  A = np.exp(Z) / sum(np.exp(Z))
/var/folders/fy/vqqz7dk157x6lpsmjr653xrh0000gn/T/ipykernel_6542/2472023414.py:17: RuntimeWarning: invalid value encountered in divide
  A = np.exp(Z) / sum(np.exp(Z))


Iteration:  0
[1 6 6 ... 6 6 3] [8 4 6 ... 6 6 6]
0.0706949152542373
Iteration:  10
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  20
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  30
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  40
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  50
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  60
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  70
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  80
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  90
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  100
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  110
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  120
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iteration:  130
[0 0 0 ... 0 0 0] [8 4 6 ... 6 6 6]
0.09872881355932203
Iter

~98% accuracy on training set.